In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("data.csv")
print(df.shape)
df.columns = ['date', 'sales', 'stock', 'price']

df['date'] = pd.to_datetime(df['date'])

# Sort by date
df = df.sort_values('date').reset_index(drop=True)

# Visual check
print(df.head())

(937, 4)
        date  sales  stock  price
0 2014-01-01      0   4972   1.29
1 2014-01-02     70   4902   1.29
2 2014-01-03     59   4843   1.29
3 2014-01-04     93   4750   1.29
4 2014-01-05     96   4654   1.29


In [2]:
target_col = 'sales'

# Scale features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.drop(columns=['date']))  # date not needed for LSTM

# Convert to numpy array
scaled_data = np.array(scaled_data)

# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :])      # past seq_length days
        y.append(data[i+seq_length, 0])        # 'sales' is the first column after drop(date)
    return np.array(X), np.array(y)

SEQ_LENGTH = 14  # using past 14 days to predict next day
X, y = create_sequences(scaled_data, SEQ_LENGTH)

print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")


Shape of X: (923, 14, 3), Shape of y: (923,)


In [3]:
# Train: 70%, Validation: 15%, Test: 15%
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")


Train: (646, 14, 3), Val: (138, 14, 3), Test: (139, 14, 3)


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# ✅ Convert data to PyTorch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# ✅ Create DataLoader for batching
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# ✅ Define LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Take last time step
        out = self.fc(out)
        return out

# Model parameters
input_size = X_train.shape[2]   # Number of features
hidden_size = 64
num_layers = 1

model = LSTMModel(input_size, hidden_size, num_layers)

# ✅ Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# ✅ Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(train_loader):.4f}")


Epoch [1/50], Loss: 0.0255
Epoch [2/50], Loss: 0.0160
Epoch [3/50], Loss: 0.0155
Epoch [4/50], Loss: 0.0150
Epoch [5/50], Loss: 0.0152
Epoch [6/50], Loss: 0.0146
Epoch [7/50], Loss: 0.0138
Epoch [8/50], Loss: 0.0133
Epoch [9/50], Loss: 0.0133
Epoch [10/50], Loss: 0.0129
Epoch [11/50], Loss: 0.0124
Epoch [12/50], Loss: 0.0122
Epoch [13/50], Loss: 0.0172
Epoch [14/50], Loss: 0.0119
Epoch [15/50], Loss: 0.0120
Epoch [16/50], Loss: 0.0118
Epoch [17/50], Loss: 0.0113
Epoch [18/50], Loss: 0.0113
Epoch [19/50], Loss: 0.0115
Epoch [20/50], Loss: 0.0120
Epoch [21/50], Loss: 0.0110
Epoch [22/50], Loss: 0.0109
Epoch [23/50], Loss: 0.0107
Epoch [24/50], Loss: 0.0107
Epoch [25/50], Loss: 0.0106
Epoch [26/50], Loss: 0.0111
Epoch [27/50], Loss: 0.0125
Epoch [28/50], Loss: 0.0108
Epoch [29/50], Loss: 0.0106
Epoch [30/50], Loss: 0.0105
Epoch [31/50], Loss: 0.0107
Epoch [32/50], Loss: 0.0104
Epoch [33/50], Loss: 0.0103
Epoch [34/50], Loss: 0.0105
Epoch [35/50], Loss: 0.0104
Epoch [36/50], Loss: 0.0103
E

In [5]:
# Put model in eval mode
model.eval()

predictions = []
actuals = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        predictions.extend(outputs.numpy())
        actuals.extend(y_batch.numpy())

predictions = np.array(predictions)
actuals = np.array(actuals)

# Metrics
mae = np.mean(np.abs(predictions - actuals))
rmse = np.sqrt(np.mean((predictions - actuals) ** 2))

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")


MAE: 0.0984
RMSE: 0.1384
